# PyCitySchools

### Dependencies and data

In [1]:
# Dependencies
import os
import numpy as np
import pandas as pd

In [2]:
# School data
school_path = os.path.join('data', 'schools.csv') # school data path
school_df = pd.read_csv(school_path)
school_df

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500
5,5,Wilson High School,Charter,2283,1319574
6,6,Cabrera High School,Charter,1858,1081356
7,7,Bailey High School,District,4976,3124928
8,8,Holden High School,Charter,427,248087
9,9,Pena High School,Charter,962,585858


In [3]:
# Student data
student_path = os.path.join('data', 'students.csv') # student data path
student_df = pd.read_csv(student_path)
student_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39170 entries, 0 to 39169
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Student ID     39170 non-null  int64 
 1   student_name   39170 non-null  object
 2   gender         39170 non-null  object
 3   grade          39170 non-null  object
 4   school_name    39170 non-null  object
 5   reading_score  39170 non-null  int64 
 6   math_score     39170 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 2.1+ MB


In [4]:
# Merge data
df = pd.merge(student_df, school_df, on='school_name', how='left')
df.head(3)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635


### Clean student names

In [5]:
# Prefixes and suffixes
student_names = [n.split() for n in student_df['student_name'].tolist() if len(n.split()) > 2]
pre = list(set([name[0] for name in student_names if len(name[0]) <= 4])) # prefixes
suf = list(set([name[-1] for name in student_names if len(name[-1]) <= 4])) # suffixes
print(pre, suf)

['Lynn', 'Leah', 'Troy', 'Kari', 'Kyle', 'Toni', 'Tara', 'Luke', 'Ms.', 'Marc', 'Jodi', 'Dana', 'Cody', 'Joe', 'Omar', 'Kara', 'Dale', 'Carl', 'Erik', 'John', 'Anna', 'Jose', 'Adam', 'Mary', 'Ryan', 'Gina', 'Amy', 'Dr.', 'Eric', 'Anne', 'Dawn', 'Todd', 'Gary', 'Gail', 'Mrs.', 'Jon', 'Lori', 'Greg', 'Noah', 'Sara', 'Jill', 'Mike', 'Judy', 'Ian', 'Mark', 'Miss', 'Tony', 'Ruth', 'Chad', 'Cory', 'Kim', 'Mr.', 'Paul', 'Emma', 'Erin', 'Sean', 'Lisa', 'Tina', 'Seth'] ['IV', 'Page', 'Pham', 'Shea', 'DDS', 'Odom', 'III', 'Cox', 'Cain', 'Dyer', 'Hood', 'Lowe', 'King', 'MD', 'West', 'Kemp', 'Pace', 'Hays', 'Lee', 'Moss', 'Bell', 'Rose', 'Holt', 'Hall', 'Chen', 'II', 'PhD', 'Sosa', 'Ross', 'Duke', 'DVM', 'Jr.', 'Ryan', 'Hale', 'Roy', 'Cole', 'Cobb', 'Moon', 'York', 'Webb', 'Park', 'Ford', 'Levy', 'Koch', 'V', 'Hill', 'Ware', 'Cook', 'Gill', 'Tate', 'Tran', 'Vang', 'Neal', 'Day', 'Dunn', 'Bond', 'Love', 'Kim', 'Diaz', 'Wood', 'Li', 'Gray', 'Mays', 'Reed']


In [6]:
# Prefixes to remove: "Miss ", "Dr. ", "Mr. ", "Ms. ", "Mrs. "
# Suffixes to remove: " MD", " DDS", " DVM", " PhD"
fixes_to_remove = ['Miss ', '\w+\. ', ' [DMP]\w?[DMS]'] # regex for prefixes and suffixes
str_to_remove = r'|'.join(fixes_to_remove) # join into a single raw str

# Remove inappropriate prefixes and suffixes
student_df['student_name'] = student_df['student_name'].str.replace(str_to_remove, '', regex=True)

In [7]:
# Check prefixes and suffixes again
student_names = [n.split() for n in student_df['student_name'].tolist() if len(n.split()) > 2]
pre = list(set([name[0] for name in student_names if len(name[0]) <= 4])) # prefixes
suf = list(set([name[-1] for name in student_names if len(name[-1]) <= 4])) # suffixes
print(pre, suf)

['Juan', 'Erik', 'Todd', 'Cory', 'Jon', 'Adam', 'Greg', 'Noah', 'Ryan', 'Cody', 'Omar', 'Mark', 'Eric', 'Sean', 'Tony', 'Seth'] ['IV', 'II', 'V', 'III', 'Jr.']


### District summary

In [8]:
# Add binary vars for passing score
df['pass_read'] = (df.reading_score >= 70).astype(int) # passing reading score
df['pass_math'] = (df.math_score >= 70).astype(int) # passing math score
df['pass_both'] = np.min([df.pass_read, df.pass_math], axis=0) # passing both scores
df.head(3)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget,pass_read,pass_math,pass_both
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635,0,1,0
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635,1,0,0
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635,1,0,0


In [9]:
# District summary
district_summary = pd.DataFrame(school_df[['size', 'budget']].sum(), columns=['District']).T
district_summary['Total Schools'] = school_df.shape[0]
district_summary = district_summary[['Total Schools', 'size', 'budget']]
district_summary_cols = ['Total Schools', 'Total Students', 'Total Budget']
district_summary

,Total Schools,size,budget
District,15,39170,24649428


In [10]:
# Score cols
score_cols = ['reading_score', 'math_score', 'pass_read', 'pass_math', 'pass_both']
score_cols_new = ['Average Reading Score', 'Average Math Score', '% Passing Reading', '% Passing Math', '% Passing Overall']

# District scores
district_scores = df[score_cols].mean()
district_scores

reading_score    81.877840
math_score       78.985371
pass_read         0.858055
pass_math         0.749809
pass_both         0.651723
dtype: float64

In [11]:
# Add scores to district summary
for col, val in district_scores.items():
    if 'pass' in col:
        val *= 100
    district_summary[col] = val
    
# Rename cols
district_summary.columns = district_summary_cols + score_cols_new
district_summary

,Total Schools,Total Students,Total Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Passing Overall
District,15,39170,24649428,81.87784,78.985371,85.805463,74.980853,65.172326


In [12]:
# Format columns
for col in district_summary.columns:
    if 'Total' in col:
        district_summary[col] = district_summary[col].apply('{:,}'.format)
    if 'Average' in col:
        district_summary[col] = district_summary[col].round(2)
    if '%' in col:
        district_summary[col] = district_summary[col].round().astype(int)
        
district_summary

,Total Schools,Total Students,Total Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Passing Overall
District,15,"39,170","24,649,428",81.88,78.99,86,75,65


### School summary

In [13]:
# Add budget per student var
school_df['budget_per_student'] = (school_df['budget'] / school_df['size']).round().astype(int)
df = pd.merge(df, school_df[['school_name', 'budget_per_student']], on='school_name', how='left')
df.head(3)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget,pass_read,pass_math,pass_both,budget_per_student
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635,0,1,0,655
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635,1,0,0,655
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635,1,0,0,655


In [14]:
# School cols
school_cols = ['type', 'size', 'budget', 'budget_per_student', 
               'reading_score', 'math_score', 'pass_read', 'pass_math', 'pass_both']
school_cols_new = ['School Type', 'Total Students', 'Total Budget', 'Budget Per Student']
school_cols_new += score_cols_new

# School summary
school_summary = df.groupby('school_name')[school_cols].agg({
    'type': 'max',
    'size': 'max',
    'budget': 'max',
    'budget_per_student': 'max',
    'reading_score': 'mean',
    'math_score': 'mean',
    'pass_read': 'mean',
    'pass_math': 'mean',
    'pass_both': 'mean'
})
school_summary.head(3)

,type,size,budget,budget_per_student,reading_score,math_score,pass_read,pass_math,pass_both
school_name,,,,,,,,,
Bailey High School,District,4976,3124928,628,81.033963,77.048432,0.819333,0.666801,0.546423
Cabrera High School,Charter,1858,1081356,582,83.975780,83.061895,0.970398,0.941335,0.913348
Figueroa High School,District,2949,1884411,639,81.158020,76.711767,0.807392,0.659885,0.532045


In [15]:
# Rename cols
school_summary.index.name = None
school_summary.columns = school_cols_new

# Format values
for col in school_summary.columns:
    if 'Total' in col:
        school_summary[col] = school_summary[col].apply('{:,}'.format)
    if 'Average' in col:
        school_summary[col] = school_summary[col].round(2)
    if '%' in col:
        school_summary[col] = (school_summary[col] * 100).round().astype(int)
        
school_summary

,School Type,Total Students,Total Budget,Budget Per Student,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Passing Overall
Bailey High School,District,"4,976","3,124,928",628,81.03,77.05,82,67,55
Cabrera High School,Charter,"1,858","1,081,356",582,83.98,83.06,97,94,91
Figueroa High School,District,"2,949","1,884,411",639,81.16,76.71,81,66,53
Ford High School,District,"2,739","1,763,916",644,80.75,77.10,79,68,54
Griffin High School,Charter,"1,468","917,500",625,83.82,83.35,97,93,91
Hernandez High School,District,"4,635","3,022,020",652,80.93,77.29,81,67,54
Holden High School,Charter,427,"248,087",581,83.81,83.80,96,93,89
Huang High School,District,"2,917","1,910,635",655,81.18,76.63,81,66,54
Johnson High School,District,"4,761","3,094,650",650,80.97,77.07,81,66,54
Pena High School,Charter,962,"585,858",609,84.04,83.84,96,95,91


### Average scores by grade

In [16]:
# Reading scores by grade of each school
grade_read_scores = pd.pivot_table(df, index='school_name', columns='grade', 
                                   values='reading_score', aggfunc='mean').round(2)
grade_read_scores.index.name = None
grade_read_scores.columns.name = 'Reading scores'
grade_read_scores = grade_read_scores[['9th', '10th', '11th', '12th']]
grade_read_scores

Reading scores,9th,10th,11th,12th
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23
Pena High School,83.81,83.61,84.34,84.59


In [17]:
# Math scores by grade of each school
grade_math_scores = pd.pivot_table(df, index='school_name', columns='grade', 
                                   values='math_score', aggfunc='mean').round(2)
grade_math_scores.index.name = None
grade_math_scores.columns.name = 'Math Scores'
grade_math_scores = grade_math_scores[['9th', '10th', '11th', '12th']]
grade_math_scores

Math Scores,9th,10th,11th,12th
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86
Pena High School,83.63,83.37,84.33,84.12


### Scores by budget per student

In [18]:
# Bin budget per student
school_df['spending_lvl'] = pd.qcut(school_df['budget_per_student'], 4, 
                                    labels=range(1, 5))
school_df['spending_lvl'].value_counts()

1    4
2    4
4    4
3    3
Name: spending_lvl, dtype: int64

In [19]:
# Bin school size
school_df['school_size'] = pd.qcut(school_df['size'], 3, labels=['Small', 'Medium', 'Large'])
school_df['school_size'].value_counts()

Small     5
Medium    5
Large     5
Name: school_size, dtype: int64

In [20]:
# Merge bin vars into combined data
df = pd.merge(df, school_df[['school_name', 'spending_lvl', 'school_size']], 
              on='school_name', how='left')
df.head(3)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget,pass_read,pass_math,pass_both,budget_per_student,spending_lvl,school_size
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635,0,1,0,655,4,Medium
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635,1,0,0,655,4,Medium
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635,1,0,0,655,4,Medium


In [21]:
# Scores by spending
spending_scores = df.groupby('spending_lvl')[score_cols].mean().round(2)
for col in spending_scores.columns:
    if "pass" in col:
        spending_scores[col] = (spending_scores[col] * 100).astype(int)
spending_scores

,reading_score,math_score,pass_read,pass_math,pass_both
spending_lvl,,,,,
1,83.96,83.36,97,94,91
2,82.31,79.98,89,79,71
3,81.48,78.05,84,71,60
4,80.96,77.06,81,67,54


In [22]:
# Formatting
spending_scores.index.name = 'Spending Level'
spending_scores.columns = score_cols_new
spending_scores

,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Passing Overall
Spending Level,,,,,
1,83.96,83.36,97,94,91
2,82.31,79.98,89,79,71
3,81.48,78.05,84,71,60
4,80.96,77.06,81,67,54


### Scores by school size

In [23]:
# Scores by school size
size_scores = df.groupby('school_size')[score_cols].mean().round(2)
for col in size_scores.columns:
    if "pass" in col:
        size_scores[col] = (size_scores[col] * 100).astype(int)
size_scores

,reading_score,math_score,pass_read,pass_math,pass_both
school_size,,,,,
Small,83.83,83.48,97,94,90
Medium,82.51,80.17,89,81,73
Large,80.96,77.02,81,66,54


In [24]:
# Formatting
size_scores.index.name = 'School Size'
size_scores.columns = score_cols_new
size_scores

,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Passing Overall
School Size,,,,,
Small,83.83,83.48,97,94,90
Medium,82.51,80.17,89,81,73
Large,80.96,77.02,81,66,54


### Scores by school type

In [25]:
# Scores by school type
type_scores = df.groupby('type')[score_cols].mean().round(2)
for col in type_scores.columns:
    if "pass" in col:
        type_scores[col] = (type_scores[col] * 100).astype(int)
type_scores

,reading_score,math_score,pass_read,pass_math,pass_both
type,,,,,
Charter,83.90,83.41,97,94,91
District,80.96,76.99,81,67,54


In [26]:
# Formatting
type_scores.index.name = 'School Type'
type_scores.columns = score_cols_new
type_scores

,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Passing Overall
School Type,,,,,
Charter,83.90,83.41,97,94,91
District,80.96,76.99,81,67,54


### Save data

In [27]:
# Save combined data
data_path = os.path.join('data', 'complete.csv')
df.to_csv(data_path, index=False)
pd.read_csv(data_path).head(3)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget,pass_read,pass_math,pass_both,budget_per_student,spending_lvl,school_size
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635,0,1,0,655,4,Medium
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635,1,0,0,655,4,Medium
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635,1,0,0,655,4,Medium
